In [1]:
import os
import pandas as pd
import datetime
import glob
from google.colab import files
import numpy as np
import re
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from tqdm import tqdm

#NO EJECUTAR --> CARGA TOTAL NO NECESARIO SI SE TIENE EL ARVCHIVO

In [2]:
i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
for year in range(2022, 2019, -1):
    for month, month_name in i2m:
        if (month > 5) and (year>2023): continue
        #Descarrega arxius de la web de Bicing
        os.system(f'curl -L -o "{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z" "https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"')
        #Extreu fitxers del format 7z
        os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        #Elimina l'arxiu comprimit
        os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")


In [3]:
import os
import re
fileList = os.listdir()

# Regex que coincideix amb: 2020_02_Febrer_BicingNou_ESTACIONS.csv
pattern = re.compile(r"^(202[0-3])_\d{2}_[A-Za-zÀ-ÿ]+_BicingNou_ESTACIONS\.csv$")

listCsv = [file for file in fileList if file.endswith(".csv") and pattern.match(file)]
listCsv = sorted(listCsv)
listCsv

['2020_01_Gener_BicingNou_ESTACIONS.csv',
 '2020_02_Febrer_BicingNou_ESTACIONS.csv',
 '2020_03_Marc_BicingNou_ESTACIONS.csv',
 '2020_04_Abril_BicingNou_ESTACIONS.csv',
 '2020_05_Maig_BicingNou_ESTACIONS.csv',
 '2020_06_Juny_BicingNou_ESTACIONS.csv',
 '2020_07_Juliol_BicingNou_ESTACIONS.csv',
 '2020_08_Agost_BicingNou_ESTACIONS.csv',
 '2020_09_Setembre_BicingNou_ESTACIONS.csv',
 '2020_10_Octubre_BicingNou_ESTACIONS.csv',
 '2020_11_Novembre_BicingNou_ESTACIONS.csv',
 '2020_12_Desembre_BicingNou_ESTACIONS.csv',
 '2021_01_Gener_BicingNou_ESTACIONS.csv',
 '2021_02_Febrer_BicingNou_ESTACIONS.csv',
 '2021_03_Març_BicingNou_ESTACIONS.csv',
 '2021_04_Abril_BicingNou_ESTACIONS.csv',
 '2021_05_Maig_BicingNou_ESTACIONS.csv',
 '2021_06_Juny_BicingNou_ESTACIONS.csv',
 '2021_07_Juliol_BicingNou_ESTACIONS.csv',
 '2021_08_Agost_BicingNou_ESTACIONS.csv',
 '2021_09_Setembre_BicingNou_ESTACIONS.csv',
 '2021_10_Octubre_BicingNou_ESTACIONS.csv',
 '2021_11_Novembre_BicingNou_ESTACIONS.csv',
 '2021_12_Desembr

In [4]:
import dask.dataframe as dd
import os
from dask.diagnostics import ProgressBar

def transform_bike_data_dask(listCsv):
    dfs = []
    for file in listCsv:
        print(f"Loading {file}...")
        ddf = dd.read_csv(
            file,
            dtype={
                "station_id": str,
                "last_reported": "float64",
                "is_returning": "object"
            },
            assume_missing=True
        )
        ddf["source_file"] = file  # opcional, para trazabilidad
        dfs.append(ddf)

    # Unir todos los CSVs en un único Dask DataFrame
    ddf = dd.concat(dfs, axis=0)

    # Procesamiento
    ddf["last_updated"] = dd.to_datetime(ddf["last_updated"], unit="s")
    # Usar 'h' en lugar de 'H' para evitar la advertencia futura
    ddf["last_updated_hour"] = ddf["last_updated"].dt.floor("h")

    ddf["total_docks"] = ddf["num_bikes_available"] + ddf["num_docks_available"]

    #Como en algunas rows del dataset la suma de "num_bikes_available" + "num_docks_available" es 0 (algo no lógico), este código mira para un station_id cual es el numero más frecuente de Docks para distintos horas, días y meses y se queda con el número mayoritario.
    most_frequent_docks = ddf.groupby("station_id")["total_docks"].apply(lambda x: x.mode()[0]).compute()

    # Esta función imputa el número de docks para aquellos casos en que tanto el número de bicis disponibles y docks sea 0 (casos en los que se genera NaN)
    def impute_docks(df, most_frequent_docks):
        for station_id, frequent_docks in most_frequent_docks.items():
            mask = (df["station_id"] == station_id) & (df["num_bikes_available"] == 0) & (df["num_docks_available"] == 0)
            df.loc[mask, "num_docks_available"] = frequent_docks - df.loc[mask, "num_bikes_available"]
        return df

    # Inocamos la función e imputamos los valores sustitutivos para "num_docks_available" y "num_bikes_available"
    ddf = ddf.map_partitions(impute_docks, most_frequent_docks=most_frequent_docks)

    ddf["percentage_docks_available"] = ddf["num_docks_available"] / ddf["total_docks"]

    # Agrupar por estación y hora
    grouped = (
        ddf.groupby(["station_id", "last_updated_hour"])
        .mean(numeric_only=True)
        .reset_index()
    )

    # Extraer componentes temporales
    grouped["month"] = grouped["last_updated_hour"].dt.month
    grouped["day"] = grouped["last_updated_hour"].dt.day
    grouped["hour"] = grouped["last_updated_hour"].dt.hour

    # Forzar el cómputo para trabajar con Pandas y evitar problemas de índice
    with ProgressBar():
        grouped_pd = grouped.compute()

    # Ordenar por station_id y last_updated_hour para aplicar shift correctamente
    grouped_pd = grouped_pd.sort_values(["station_id", "last_updated_hour"]).reset_index(drop=True)

    # Calcular ctx-0 (valor actual)
    grouped_pd["ctx-0"] = grouped_pd["percentage_docks_available"]

    # Calcular ctx-1 a ctx-4 de forma continua por station_id
    for i in range(1, 5):
        grouped_pd[f"ctx-{i}"] = (
            grouped_pd.groupby("station_id")["percentage_docks_available"]
            .shift(i)
            .fillna(0)
        )

    # Seleccionar columnas finales
    final_cols = [
        "station_id", "month", "day", "hour",
        "ctx-4", "ctx-3", "ctx-2", "ctx-1", "ctx-0"
    ]
    final_df = grouped_pd[final_cols]

    return final_df

1. Carregar tots els fitxers CSV en dataframes i, a continuació, calcular la mitjana per hora de les variables (fem la mitjana per a cada estació station_id i hora), intentem usar Dask però no és l'òptim per a nosaltres. A més afegim la columna de mes (month_num) i any (year) per quan unim tots els diferents mesos saber d'on provenien les observacions, i les variables 'ctx-4', 'ctx-3', 'ctx-2', 'ctx-1', 'percentage_docks_available', 'percentage_bikes_mech', 'percentage_bikes_e'.

In [ ]:
listCsv = sorted(listCsv)

df_result = transform_bike_data_dask(listCsv)

Loading 2020_01_Gener_BicingNou_ESTACIONS.csv...
Loading 2020_02_Febrer_BicingNou_ESTACIONS.csv...
Loading 2020_03_Marc_BicingNou_ESTACIONS.csv...
Loading 2020_04_Abril_BicingNou_ESTACIONS.csv...
Loading 2020_05_Maig_BicingNou_ESTACIONS.csv...
Loading 2020_06_Juny_BicingNou_ESTACIONS.csv...
Loading 2020_07_Juliol_BicingNou_ESTACIONS.csv...
Loading 2020_08_Agost_BicingNou_ESTACIONS.csv...
Loading 2020_09_Setembre_BicingNou_ESTACIONS.csv...
Loading 2020_10_Octubre_BicingNou_ESTACIONS.csv...
Loading 2020_11_Novembre_BicingNou_ESTACIONS.csv...
Loading 2020_12_Desembre_BicingNou_ESTACIONS.csv...
Loading 2021_01_Gener_BicingNou_ESTACIONS.csv...
Loading 2021_02_Febrer_BicingNou_ESTACIONS.csv...
Loading 2021_03_Març_BicingNou_ESTACIONS.csv...
Loading 2021_04_Abril_BicingNou_ESTACIONS.csv...
Loading 2021_05_Maig_BicingNou_ESTACIONS.csv...
Loading 2021_06_Juny_BicingNou_ESTACIONS.csv...
Loading 2021_07_Juliol_BicingNou_ESTACIONS.csv...
Loading 2021_08_Agost_BicingNou_ESTACIONS.csv...
Loading 202

<ipython-input-4-2c933a06df1b>:32: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  most_frequent_docks = ddf.groupby("station_id")["total_docks"].apply(lambda x: x.mode()[0]).compute()


In [ ]:
df_result.to_csv("df_result_with_year.csv", index=False)


# EJECUTAR DESDE AQUI

## LOAD BIG DATASET, NAS AND CREATE X,Y FOR TRAIN

In [ ]:
#https://www.kaggle.com/competitions/2025-bike-availability-prediction/data?select=metadata_sample_submission_2025.csv

In [ ]:
import pandas as pd

df = pd.read_csv('df_result_with_year.csv')

print(df)

        station_id  year  month  day  hour     ctx-4     ctx-3     ctx-2  \
0                1  2019     12   31    23  0.000000  0.000000  0.000000   
1                1  2020      1    1     0  0.000000  0.000000  0.000000   
2                1  2020      1    1     1  0.000000  0.000000  0.509470   
3                1  2020      1    1     2  0.000000  0.509470  0.469697   
4                1  2020      1    1     3  0.509470  0.469697  0.407343   
...            ...   ...    ...  ...   ...       ...       ...       ...   
920276         131  2020     12   19     5  0.250000  0.250000  0.263889   
920277         131  2020     12   19     6  0.250000  0.263889  0.333333   
920278         131  2020     12   19     7  0.263889  0.333333  0.385417   
920279         131  2020     12   19     8  0.333333  0.385417  0.375000   
920280         131  2020     12   19     9  0.385417  0.375000  0.364583   

           ctx-1     ctx-0  
0       0.000000  0.509470  
1       0.509470  0.469697  


In [ ]:
# Contar valores NaN por columna
print("Valores NaN por columna:")
print(df.isnull().sum())

# Contar valores 0 por columna (ten en cuenta que para columnas numéricas)
print("\nValores 0 por columna:")
print((df == 0).sum())

# Eliminar filas con NaNs
df = df.dropna()

# Verificar que se han eliminado los NaNs
print("\nValores NaN por columna después de limpiar:")
print(df.isnull().sum())



Valores NaN por columna:
station_id      0
year            0
month           0
day             0
hour            0
ctx-4           0
ctx-3           0
ctx-2           0
ctx-1           0
ctx-0         784
dtype: int64

Valores 0 por columna:
station_id        0
year              0
month             0
day               0
hour          38141
ctx-4          8084
ctx-3          8047
ctx-2          8011
ctx-1          7974
ctx-0          7155
dtype: int64

Valores NaN por columna después de limpiar:
station_id    0
year          0
month         0
day           0
hour          0
ctx-4         0
ctx-3         0
ctx-2         0
ctx-1         0
ctx-0         0
dtype: int64


In [ ]:
df=df[df['year']>=2021]

In [ ]:
X_train = df.iloc[:, :9]
X_train.columns

Index(['station_id', 'year', 'month', 'day', 'hour', 'ctx-4', 'ctx-3', 'ctx-2',
       'ctx-1'],
      dtype='object')

In [ ]:
y_train = df.iloc[:, 9:]
y_train.columns

Index(['ctx-0'], dtype='object')

## LOAD X TEST

In [ ]:
import pandas as pd

# Ruta al archivo CSV
ruta_csv = "metadata_sample_submission_2025.csv"

# Carga del CSV en un DataFrame
X_test = pd.read_csv(ruta_csv)

# Visualizar las primeras filas
print(X_test.head())

   index  station_id  month  day  hour     ctx-4     ctx-3     ctx-2     ctx-1
0      0           1      6    1     3  0.490942  0.378623  0.324275  0.311594
1      1           1      6    1     8  0.271739  0.311594  0.346014  0.394928
2      2           1      6    1    13  0.538043  0.650362  0.697464  0.721014
3      3           1      6    1    18  0.789855  0.800725  0.791667  0.807971
4      4           1      6    1    23  0.860507  0.871377  0.817029  0.793478


In [ ]:
X_test['year'] = 2024


In [ ]:
X_test

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year
0,0,1,6,1,3,0.490942,0.378623,0.324275,0.311594,2024
1,1,1,6,1,8,0.271739,0.311594,0.346014,0.394928,2024
2,2,1,6,1,13,0.538043,0.650362,0.697464,0.721014,2024
3,3,1,6,1,18,0.789855,0.800725,0.791667,0.807971,2024
4,4,1,6,1,23,0.860507,0.871377,0.817029,0.793478,2024
...,...,...,...,...,...,...,...,...,...,...
401506,401506,496,12,31,2,0.865741,0.643519,0.597222,0.921296,2024
401507,401507,496,12,31,7,1.000000,0.388889,0.375000,0.407407,2024
401508,401508,496,12,31,12,0.310185,0.337963,0.402778,0.495370,2024
401509,401509,496,12,31,17,0.462963,0.564815,0.629630,0.583333,2024


## WEATHER DATA


In [ ]:
import pandas as pd

# Ruta al archivo CSV
meteo_csv = "meteov2.csv"

# Carga del CSV en un DataFrame
meteo = pd.read_csv(meteo_csv)

# Visualizar las primeras filas
print(meteo.head())

   tavg  prcp  snow  year  month  day  hour   wind  wind_flag
0   7.9     1     0  2020      1    1     0  21.85          0
1   7.8     1     0  2020      1    2     0  22.15          0
2   7.1     1     0  2020      1    3     0  23.60          0
3   7.9     1     0  2020      1    4     0  26.95          0
4   8.5     1     0  2020      1    5     0  28.25          0


In [ ]:
meteo = pd.read_csv(meteo_csv)


In [ ]:
meteo= meteo.drop(columns=['hour','wind_flag'])

In [ ]:
meteo

,tavg,prcp,snow,year,month,day,wind
0,7.9,1,0,2020,1,1,21.85
1,7.8,1,0,2020,1,2,22.15
2,7.1,1,0,2020,1,3,23.60
3,7.9,1,0,2020,1,4,26.95
4,8.5,1,0,2020,1,5,28.25
...,...,...,...,...,...,...,...
1905,13.1,0,0,2025,3,20,35.35
1906,13.1,0,0,2025,3,21,30.95
1907,12.2,1,0,2025,3,22,31.40
1908,11.6,1,0,2025,3,23,24.15


In [ ]:
df_merged = pd.merge(df, meteo, on=['year', 'month', 'day'], how='inner')
df_merged

,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,ctx-0,tavg,prcp,snow,wind
0,1,2021,1,1,0,0.043478,0.045290,0.050725,0.063406,0.119565,7.2,1,0,20.35
1,1,2021,1,1,1,0.045290,0.050725,0.063406,0.119565,0.108696,7.2,1,0,20.35
2,1,2021,1,1,2,0.050725,0.063406,0.119565,0.108696,0.108696,7.2,1,0,20.35
3,1,2021,1,1,3,0.063406,0.119565,0.108696,0.108696,0.108696,7.2,1,0,20.35
4,1,2021,1,1,4,0.119565,0.108696,0.108696,0.108696,0.108696,7.2,1,0,20.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625959,130,2022,12,31,19,0.260417,0.190972,0.256944,0.593750,0.333333,14.2,1,0,18.90
625960,130,2022,12,31,20,0.190972,0.256944,0.593750,0.333333,0.309028,14.2,1,0,18.90
625961,130,2022,12,31,21,0.256944,0.593750,0.333333,0.309028,0.416667,14.2,1,0,18.90
625962,130,2022,12,31,22,0.593750,0.333333,0.309028,0.416667,0.458333,14.2,1,0,18.90


In [ ]:
X_train = df_merged.drop(columns=['ctx-0','year'])
y_train = df_merged['ctx-0']


In [ ]:
X_test=pd.merge(X_test, meteo, on=['year', 'month', 'day'], how='inner')
X_test

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year,tavg,prcp,snow,wind
0,0,1,6,1,3,0.490942,0.378623,0.324275,0.311594,2024,18.7,1,0,24.80
1,1,1,6,1,8,0.271739,0.311594,0.346014,0.394928,2024,18.7,1,0,24.80
2,2,1,6,1,13,0.538043,0.650362,0.697464,0.721014,2024,18.7,1,0,24.80
3,3,1,6,1,18,0.789855,0.800725,0.791667,0.807971,2024,18.7,1,0,24.80
4,4,1,6,1,23,0.860507,0.871377,0.817029,0.793478,2024,18.7,1,0,24.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401506,401506,496,12,31,2,0.865741,0.643519,0.597222,0.921296,2024,7.6,1,0,21.65
401507,401507,496,12,31,7,1.000000,0.388889,0.375000,0.407407,2024,7.6,1,0,21.65
401508,401508,496,12,31,12,0.310185,0.337963,0.402778,0.495370,2024,7.6,1,0,21.65
401509,401509,496,12,31,17,0.462963,0.564815,0.629630,0.583333,2024,7.6,1,0,21.65


In [ ]:
# Eliminar la columna "Unnamed: 0" de X_train
#X_train = X_train.drop(columns=['Unnamed: 0','year'])
X_test = X_test.drop(columns=['year'])


# Establecer la columna "index" como índice en X_test


In [ ]:
X_test = X_test.set_index('index')


# MODELS

## LINEAR REGRESSION

In [ ]:
simple_model = LinearRegression()
simple_model.fit(X_train, y_train)

In [ ]:
y_pred = simple_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test.index
})


df_output


In [ ]:
df_output.to_csv("submit_weath_v3.csv", index=False)


## RED NEURONAL

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
from google.colab import files
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Escalar y aplicar PCA a X_train
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

pca = PCA(n_components=0.95)  # Mantener el 95% de la varianza explicada
X_train_scaled = pca.fit_transform(X_train_scaled)

# Transformar X_test con el mismo scaler y PCA
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)

# Crear y compilar el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

# Optimizador con decaimiento exponencial del learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Callback: parar si no mejora tras 3 épocas
early_stop = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)

# Entrenar el modelo
model.fit(
    X_train_scaled,
    y_train,
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# Predicción sobre X_test
y_pred = model.predict(X_test_pca)
print(y_pred)


#RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_RF = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model_RF.fit(X_train, y_train)


# XGRADIENT BOSTING

In [ ]:
from xgboost import XGBRegressor

model_X = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6)
model_X.fit(X_train, y_train)

In [ ]:
y_pred = model_X.predict(X_test)

In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test.index
})


df_output
df_output.to_csv("submit_weath_v4_X.csv", index=False)


# XGRADIENT BOSTING 2

In [ ]:
from xgboost import XGBRegressor

model_X = XGBRegressor(n_estimators=350, learning_rate=0.05, max_depth=10)
model_X.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=350, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = model_X.predict(X_test)

In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test.index
})


df_output
df_output.to_csv("submit_weath_v7_X.csv", index=False)


# OTRO

In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.871045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1822
[LightGBM] [Info] Number of data points in the train set: 12674966, number of used features: 11
[LightGBM] [Info] Start training from score 0.630710


LGBMRegressor(max_depth=6, random_state=42)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test.index
})


df_output
df_output.to_csv("submit_weath_v5_LGM.csv", index=False)
